# Extract the Training Features for the September 2021 to January 2022 season

In [1]:
import os

import geopandas as gpd
import pandas as pd
from deafrica_tools.classification import collect_training_data
from feature_collection import feature_layers
from odc.io.cgroups import get_cpu_quota
from sklearn.model_selection import train_test_split

In [2]:
# Define the analysis parameters.
output_crs = "EPSG:6933"
resolution = (-10, 10)
time_range = ("2021-09", "2022-01")

# Column in the training data containing the numeric class labels.
field = "label"

# Define the path to save the training features to.
output_folder="results"

# Define the path to the training data.
training_data_file = "clean_data/sep_2021_to_jan_2022_training_data.geojson"

# fraction of the dataset will be set aside as the testing dataset.
test_size = 0.20

# random state
random_state = 42

In [3]:
# Create the results directory if it does not exist.
os.makedirs(output_folder, exist_ok=True)

In [4]:
# Load the  training data
training_data = gpd.read_file(training_data_file).to_crs(output_crs)

In [5]:
# Create a datacube query object using our analysis parameters.
query = dict(output_crs=output_crs, resolution=resolution, time=time_range)
query

{'output_crs': 'EPSG:6933',
 'resolution': (-10, 10),
 'time': ('2021-09', '2022-01')}

In [6]:
# Set parallel mode on or off (set to False if testing a new feature extraction function).
ncpus = round(get_cpu_quota())
print("ncpus = " + str(ncpus))

ncpus = 15


In [7]:
%%time
column_names, model_input = collect_training_data(
                                    gdf=training_data,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    zonal_stats=None,
                                    feature_func=feature_layers
                                    )

  0%|          | 0/70 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning

Percentage of possible fails after run 1 = 0.0 %
Removed 0 rows wth NaNs &/or Infs
Output shape:  (2857, 91)
CPU times: user 334 ms, sys: 146 ms, total: 480 ms
Wall time: 2min 29s


In [8]:
# Convert to dataframe.
model_input_df = pd.DataFrame(model_input, columns=column_names)

In [9]:
label_col = model_input_df.columns[0]
feature_cols = model_input_df.columns[1:]

# Split the data into features and labels
X = model_input_df[feature_cols].values
y = model_input_df[label_col].values.ravel()

In [10]:
# Print the list of features collected
print("# of training features collected:", len(feature_cols))
print("List of training features collected:", feature_cols)

# of training features collected: 90
List of training features collected: Index(['vv_2021-09-30', 'vv_2021-10-31', 'vv_2021-11-30', 'vv_2021-12-31',
       'vv_2022-01-31', 'vh_2021-09-30', 'vh_2021-10-31', 'vh_2021-11-30',
       'vh_2021-12-31', 'vh_2022-01-31', 'red_2021-09-30', 'red_2021-10-31',
       'red_2021-11-30', 'red_2021-12-31', 'red_2022-01-31',
       'green_2021-09-30', 'green_2021-10-31', 'green_2021-11-30',
       'green_2021-12-31', 'green_2022-01-31', 'blue_2021-09-30',
       'blue_2021-10-31', 'blue_2021-11-30', 'blue_2021-12-31',
       'blue_2022-01-31', 'red_edge_1_2021-09-30', 'red_edge_1_2021-10-31',
       'red_edge_1_2021-11-30', 'red_edge_1_2021-12-31',
       'red_edge_1_2022-01-31', 'red_edge_2_2021-09-30',
       'red_edge_2_2021-10-31', 'red_edge_2_2021-11-30',
       'red_edge_2_2021-12-31', 'red_edge_2_2022-01-31',
       'red_edge_3_2021-09-30', 'red_edge_3_2021-10-31',
       'red_edge_3_2021-11-30', 'red_edge_3_2021-12-31',
       'red_edge_3_2022

In [11]:
# Split the data into the training and validation dataset.
X_train, X_test, y_train, y_test= train_test_split(X, y, stratify=y, test_size=test_size, random_state=random_state)

In [12]:
# Save the training dataset to use for training the model.
training_df = pd.DataFrame(data=X_train, columns=feature_cols)
training_df[label_col] = y_train
training_df = training_df[model_input_df.columns]
training_df.to_csv(os.path.join(output_folder, 'sep_2021_to_jan_2022_training_features_training.csv'), index=False)


In [13]:
# Save the validation dataset to use for accuracy assessment.
validation_df = pd.DataFrame(data=X_test, columns=feature_cols)
validation_df[label_col] = y_test
validation_df = validation_df[model_input_df.columns]
validation_df.to_csv(os.path.join(output_folder, 'sep_2021_to_jan_2022_training_features_validation.csv'), index=False)

In [14]:
len(training_df)


2285

In [15]:
len(validation_df)


572